<a href="https://colab.research.google.com/github/gremlin97/EVA-8/blob/main/S5/EVA3_S5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from model import Net #Importing `Net` from model.py

In [ ]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Group Normalization


In [ ]:
from torchsummary import summary
norm = 'GN' #Using group norm

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net(norm,num=2).to(device) # Setting number of groups to 2
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,608
         MaxPool2d-4           [-1, 32, 12, 12]               0
           Dropout-5           [-1, 32, 12, 12]               0
            Conv2d-6           [-1, 10, 12, 12]             320
            Conv2d-7           [-1, 16, 10, 10]           1,440
           Dropout-8           [-1, 16, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           2,304
          Dropout-10             [-1, 16, 8, 8]               0
           Conv2d-11             [-1, 16, 6, 6]           2,304
AdaptiveAvgPool2d-12             [-1, 16, 1, 1]               0
           Linear-13                   [-1, 10]             170
Total params: 11,290
Trainable params: 

/content/model.py:112: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Train Accuracy={100. * correct / len(train_loader.dataset)}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
scheduler =  optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)

for epoch in range(20):
    print("Epoch: ",epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

Epoch:  1


loss=0.028912212699651718 batch_id=1874 Train Accuracy=94.15333333333334: 100%|██████████| 1875/1875 [00:36<00:00, 51.09it/s]



Test set: Average loss: 0.0543, Accuracy: 9846/10000 (98%)

Epoch:  2


loss=0.017685377970337868 batch_id=1874 Train Accuracy=97.76666666666667: 100%|██████████| 1875/1875 [00:33<00:00, 55.91it/s]



Test set: Average loss: 0.0631, Accuracy: 9805/10000 (98%)

Epoch:  3


...


Test set: Average loss: 0.0232, Accuracy: 9936/10000 (99%)

Epoch:  19


loss=0.00794703047722578 batch_id=1874 Train Accuracy=99.41833333333334: 100%|██████████| 1875/1875 [00:33<00:00, 56.71it/s]



Test set: Average loss: 0.0226, Accuracy: 9935/10000 (99%)

Epoch:  20


loss=0.002426941180601716 batch_id=1874 Train Accuracy=99.445: 100%|██████████| 1875/1875 [00:33<00:00, 56.36it/s]



Test set: Average loss: 0.0227, Accuracy: 9936/10000 (99%)



## Layer Normalization

In [ ]:
from torchsummary import summary
norm = 'LN' # Using layer norm

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
modelln = Net(norm).to(device)
summary(modelln, input_size=(1, 28, 28))

torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,608
         MaxPool2d-4           [-1, 32, 12, 12]               0
           Dropout-5           [-1, 32, 12, 12]               0
            Conv2d-6           [-1, 10, 12, 12]             320
            Conv2d-7           [-1, 16, 10, 10]           1,440
           Dropout-8           [-1, 16, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           2,304
          Dropout-10             [-1, 16, 8, 8]               0
           Conv2d-11             [-1, 16, 6, 6]           2,304
AdaptiveAvgPool2d-12             [-1, 16, 1, 1]               0
           Linear-13                   [-1, 10]             170
Total params: 11,290
Trainable params: 

In [ ]:
optimizer = optim.SGD(modelln.parameters(), lr=0.05, momentum=0.9)
scheduler =  optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)

for epoch in range(20):
    print("Epoch: ",epoch+1)
    train(modelln, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(modelln, device, test_loader)

Epoch:  1


loss=0.0783224031329155 batch_id=1874 Train Accuracy=92.38: 100%|██████████| 1875/1875 [00:32<00:00, 57.97it/s]



Test set: Average loss: 0.0643, Accuracy: 9804/10000 (98%)

Epoch:  2


loss=0.0037191607989370823 batch_id=1874 Train Accuracy=97.45666666666666: 100%|██████████| 1875/1875 [00:32<00:00, 57.88it/s]



Test set: Average loss: 0.0573, Accuracy: 9809/10000 (98%)

Epoch:  3


...


Test set: Average loss: 0.0200, Accuracy: 9940/10000 (99%)

Epoch:  19


loss=0.0031574408058077097 batch_id=1874 Train Accuracy=99.40166666666667: 100%|██████████| 1875/1875 [00:32<00:00, 57.83it/s]



Test set: Average loss: 0.0199, Accuracy: 9940/10000 (99%)

Epoch:  20


loss=0.01111519429832697 batch_id=1874 Train Accuracy=99.375: 100%|██████████| 1875/1875 [00:31<00:00, 58.87it/s]



Test set: Average loss: 0.0197, Accuracy: 9941/10000 (99%)



## BatchNorm + L1

In [ ]:
from torchsummary import summary
norm = 'BN' #Using Batch Norm + L1

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
modelbn = Net(norm).to(device)
summary(modelbn, input_size=(1, 28, 28))

torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,608
         MaxPool2d-4           [-1, 32, 12, 12]               0
           Dropout-5           [-1, 32, 12, 12]               0
            Conv2d-6           [-1, 10, 12, 12]             320
            Conv2d-7           [-1, 16, 10, 10]           1,440
           Dropout-8           [-1, 16, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           2,304
          Dropout-10             [-1, 16, 8, 8]               0
           Conv2d-11             [-1, 16, 6, 6]           2,304
AdaptiveAvgPool2d-12             [-1, 16, 1, 1]               0
           Linear-13                   [-1, 10]             170
Total params: 11,290
Trainable params: 

/content/model.py:124: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
# Function to implement L1 regularization
def trainL1(model, device, train_loader, optimizer, epoch, lambda_l1):
    model.train()
    correct = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        l1=0

        # L1 Norm
        for p in model.parameters():
          l1+=p.abs().sum()

        loss += lambda_l1*l1
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Train Accuracy={100. * correct / len(train_loader.dataset)}')

In [ ]:
optimizer = optim.SGD(modelbn.parameters(), lr=0.05, momentum=0.9)
scheduler =  optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)

for epoch in range(20):
    print("Epoch: ",epoch+1)
    trainL1(modelbn, device, train_loader, optimizer, epoch, 0.005)
    scheduler.step()
    test(modelbn, device, test_loader)

Epoch:  1


loss=1.6456964015960693 batch_id=1874 Train Accuracy=87.84166666666667: 100%|██████████| 1875/1875 [00:32<00:00, 58.22it/s]



Test set: Average loss: 0.3638, Accuracy: 8871/10000 (89%)

Epoch:  2


loss=1.6156458854675293 batch_id=1874 Train Accuracy=88.195: 100%|██████████| 1875/1875 [00:33<00:00, 56.53it/s]



Test set: Average loss: 0.3297, Accuracy: 8998/10000 (90%)

Epoch:  3


...


Test set: Average loss: 0.2218, Accuracy: 9342/10000 (93%)

Epoch:  19


loss=0.7846667766571045 batch_id=1874 Train Accuracy=95.23666666666666: 100%|██████████| 1875/1875 [00:33<00:00, 55.84it/s]



Test set: Average loss: 0.1295, Accuracy: 9633/10000 (96%)

Epoch:  20


loss=0.7183787822723389 batch_id=1874 Train Accuracy=95.01166666666667: 100%|██████████| 1875/1875 [00:33<00:00, 55.96it/s]



Test set: Average loss: 0.1202, Accuracy: 9661/10000 (97%)

